In [1]:
import sys
print(sys.executable)

C:\ProgramData\Anaconda2\envs\tf\python.exe


In [2]:
import pandas as pd

In [3]:
df = pd.read_pickle('firehose10m')

In [4]:
df.head()

,ROWNUM,LAT,LNG,COLLECT_TIME,TIMEZONEOFFSET,CATEGORY,CATEGORY_NAME,COLLECT_TIME_UTC,COLLECT_TIME_lOCAL
0,1,40.340646,36.562329,1.472502e+09,180.0,4bf58dd8d48988d165941735,Outdoors & Recreation,2016-08-29 20:12:12,2016-08-29 23:12:12
1,2,37.036869,35.314641,1.472502e+09,180.0,4bf58dd8d48988d116941735,Nightlife Spot,2016-08-29 20:12:12,2016-08-29 23:12:12
2,3,39.598118,26.973218,1.472502e+09,180.0,4bf58dd8d48988d1e0931735,Food,2016-08-29 20:12:12,2016-08-29 23:12:12
3,4,38.388492,27.165315,1.472502e+09,180.0,4bf58dd8d48988d116941735,Nightlife Spot,2016-08-29 20:12:12,2016-08-29 23:12:12
4,5,36.754580,34.538092,1.472502e+09,180.0,4bf58dd8d48988d1d0941735,Food,2016-08-29 20:12:12,2016-08-29 23:12:12


In [5]:
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import Point

In [8]:
geom = df.apply(lambda x : Point([x['LNG'],x['LAT']]), axis=1)

In [9]:
df = gpd.GeoDataFrame(df, geometry=geom)
df.crs = {'init' :'epsg:4326'}
df.head()

,ROWNUM,LAT,LNG,COLLECT_TIME,TIMEZONEOFFSET,CATEGORY,CATEGORY_NAME,COLLECT_TIME_UTC,COLLECT_TIME_lOCAL,geometry
0,1,40.340646,36.562329,1.472502e+09,180.0,4bf58dd8d48988d165941735,Outdoors & Recreation,2016-08-29 20:12:12,2016-08-29 23:12:12,POINT (36.56232940745824 40.34064632272965)
1,2,37.036869,35.314641,1.472502e+09,180.0,4bf58dd8d48988d116941735,Nightlife Spot,2016-08-29 20:12:12,2016-08-29 23:12:12,POINT (35.31464113166668 37.03686852745004)
2,3,39.598118,26.973218,1.472502e+09,180.0,4bf58dd8d48988d1e0931735,Food,2016-08-29 20:12:12,2016-08-29 23:12:12,POINT (26.97321839034275 39.59811832811547)
3,4,38.388492,27.165315,1.472502e+09,180.0,4bf58dd8d48988d116941735,Nightlife Spot,2016-08-29 20:12:12,2016-08-29 23:12:12,POINT (27.16531532459905 38.38849204824997)
4,5,36.754580,34.538092,1.472502e+09,180.0,4bf58dd8d48988d1d0941735,Food,2016-08-29 20:12:12,2016-08-29 23:12:12,POINT (34.53809185709999 36.75457974601678)


In [10]:
geojson_file = 'nycGeo.geojson'

In [11]:
TAZ = gpd.read_file(geojson_file)[['TAZ','geometry']]

In [12]:
TAZ.head()

,TAZ,geometry
0,1,"(POLYGON ((-74.01683300000001 40.705454, -74.0..."
1,2,"(POLYGON ((-74.01317400000001 40.719443, -74.0..."
2,3,"(POLYGON ((-74.027877 40.684782, -74.0278 40.6..."
3,4,"(POLYGON ((-74.008641 40.703439, -74.009623 40..."
4,5,"(POLYGON ((-74.005233 40.705619, -74.005645 40..."


In [13]:
df_and_TAZ = gpd.sjoin(df, TAZ, op='within')

In [14]:
len(df_and_TAZ)

49647

In [15]:
df_and_TAZ.head()

,ROWNUM,LAT,LNG,COLLECT_TIME,TIMEZONEOFFSET,CATEGORY,CATEGORY_NAME,COLLECT_TIME_UTC,COLLECT_TIME_lOCAL,geometry,index_right,TAZ
71,72,40.773644,-73.960819,1.472502e+09,-240.0,4bf58dd8d48988d196941735,Professional & Other Places,2016-08-29 20:12:14,2016-08-29 16:12:14,POINT (-73.96081924438477 40.77364375001149),187,188
354573,354574,40.771194,-73.961896,1.472512e+09,-240.0,4bf58dd8d48988d10c941735,Food,2016-08-29 23:07:26,2016-08-29 19:07:26,POINT (-73.961896067448 40.77119351512221),187,188
458903,458904,40.770490,-73.962803,1.472516e+09,-240.0,4bf58dd8d48988d12b951735,Travel & Transport,2016-08-30 00:08:03,2016-08-29 20:08:03,POINT (-73.96280294980241 40.77049022308537),187,188
515788,515789,40.771442,-73.961613,1.472518e+09,-240.0,4bf58dd8d48988d1cc941735,Food,2016-08-30 00:48:52,2016-08-29 20:48:52,POINT (-73.96161317825316 40.77144186567578),187,188
1641994,1641995,40.770144,-73.961526,1.472582e+09,-240.0,4bf58dd8d48988d177941735,Professional & Other Places,2016-08-30 18:27:54,2016-08-30 14:27:54,POINT (-73.96152617499997 40.77014355),187,188


In [16]:
df1=df[df['LNG'].between(-74.0516,-73.9038) & df['LAT'].between(40.6818,40.8832)].copy()

In [17]:
df1_and_TAZ = gpd.sjoin(df1, TAZ, op='within')
len(df1_and_TAZ)

49647

In [19]:
df_and_TAZ.to_pickle('firehoseTAZ')